# Modeling connected wells with the WellString element

A WellString can be used to model a series of connected wells. There are two types of
WellString:

- `WellString`: connected wells with equal head inside the wells and a user-specified total discharge
- `HeadWellString`: connected wells with equal head inside the wells and a specified head at a point

This notebook shows how to model these elements and compares the results to models with
individual wells. Examples are shown for both elements in single- and multi-layer models.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import timml as tml

## WellString

A series of wells that have equal head (pressure) inside the well and pump with a total specfied discharge.

### Single layer model

In [ ]:
# model parameters
kh = 10  # m/day

ctop = 1000.0  # resistance top leaky layer in days

ztop = 0.0  # surface elevation
zbot = -20.0  # bottom elevation of the model

z = [1.0, ztop, zbot]
kaq = np.array([kh])
c = np.array([ctop])

Reference model with 2 equal discharge wells

In [ ]:
mlref0 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
w1 = tml.Well(mlref0, 0, -10, Qw=50, rw=0.1)
w2 = tml.Well(mlref0, 0, 10, Qw=50, rw=0.1)
mlref0.solve()

Reference model with 2 head-specified wells

In [ ]:
mlref1 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
wh1 = tml.HeadWell(mlref1, 0, -10, hw=w1.headinside().item(), rw=0.1)
wh2 = tml.HeadWell(mlref1, 0, 10, hw=w1.headinside().item(), rw=0.1)
mlref1.solve()

Model with a WellString

In [ ]:
ml0 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
ws = tml.WellString(ml0, xy=[(0, -10), (0, 10)], Qw=100, rw=0.1)
ml0.solve()

Contour the heads of the first reference model and the WellString model.

In [ ]:
levels = 10
mlref0.plots.contour(
    win=(-50, 50, -50, 50), ngr=51, levels=levels, decimals=2, layers=[0]
)
ml0.plots.contour(
    win=(-50, 50, -50, 50),
    ngr=51,
    levels=levels,
    decimals=2,
    layers=[0],
    newfig=False,
    linestyles="dashed",
    color="C1",
);

Compare heads along y between all 3 models (2 reference models and the WellString model).

In [ ]:
y = np.linspace(-50, 50, 101)
x = np.zeros_like(y)
href = mlref0.headalongline(x, y)
href2 = mlref1.headalongline(x, y)
h0 = ml0.headalongline(x, y)

plt.figure(figsize=(10, 2))
plt.plot(y, href[0], label="Reference Wells")
plt.plot(y, href2[0], "--", label="Reference HeadWells")
plt.plot(y, h0[0], "k.", label="WellString model")
plt.xlabel("y [m]")
plt.ylabel("head [m]")
plt.legend(loc=(0, 1), frameon=False, ncol=3);

Check computed discharges

In [ ]:
# 2 discharge wells, Q specified not computed
w1.discharge(), w2.discharge()

In [ ]:
# 2 HeadWells
wh1.discharge(), wh2.discharge()

In [ ]:
# WellString
ws.discharge()

Compare heads inside the wells

In [ ]:
# 2 discharge wells
w1.headinside(), w2.headinside()

In [ ]:
# 2 HeadWells, specified, not computed
wh1.headinside(), wh2.headinside()

In [ ]:
# WellString
ws.wlist[0].headinside(), ws.wlist[1].headinside()

### Multilayer example

The following example compares the WellString element to reference models with
individual Wells and HeadWells.

In [ ]:
# model parameters
kh = [5, 10, 20]  # m/day

c = [1000.0, 100.0, 1.0]  # resistance leaky layers in days

ztop = 0.0  # surface elevation
zbot = -50.0  # bottom elevation of the model

z = [1.0, ztop, -10, -15, -25, -26, zbot]
kaq = np.array(kh)
c = np.array(c)

Reference model with discharge wells

In [ ]:
mlref2 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
w1 = tml.Well(mlref2, 0, -10, Qw=50, rw=0.1, layers=[1, 2])
w2 = tml.Well(mlref2, 0, 10, Qw=50, rw=0.1, layers=[1, 2])
mlref2.solve()

Reference model with head-specified wells

In [ ]:
mlref3 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
wh1 = tml.HeadWell(mlref3, 0, -10, hw=w1.headinside(), rw=0.1, layers=[1, 2])
wh2 = tml.HeadWell(mlref3, 0, 10, hw=w1.headinside(), rw=0.1, layers=[1, 2])
mlref3.solve()

Model with WellString

In [ ]:
ml1 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
ws = tml.WellString(ml1, xy=[(0, -10), (0, 10)], Qw=100, rw=0.1, layers=[1, 2])
ml1.solve()

Compare head contours

In [ ]:
ilay = 1
levels = 10
mlref2.plots.contour(
    win=(-50, 50, -50, 50),
    ngr=51,
    levels=levels,
    decimals=2,
    layers=[ilay],
)
ml1.plots.contour(
    win=(-50, 50, -50, 50),
    ngr=51,
    levels=levels,
    decimals=2,
    layers=[ilay],
    newfig=False,
    linestyles="dashed",
    color="C1",
);

Compare drawdowns along y

In [ ]:
y = np.linspace(-50, 50, 101)
x = np.zeros_like(y)
href2 = mlref2.headalongline(x, y)
href3 = mlref3.headalongline(x, y)
h1 = ml1.headalongline(x, y)

ilay = 1
plt.figure(figsize=(10, 2))
plt.plot(y, href2[ilay], label="Reference Well")
plt.plot(y, href3[ilay], "--", label="Reference HeadWell")
plt.plot(y, h1[ilay], "k.", label="WellString model")
plt.xlabel("y [m]")
plt.ylabel("head [m]")
plt.legend(loc=(0, 1), frameon=False, ncol=3);

Compare discharges

In [ ]:
# 2 discharge wells, total Q specified
w1.discharge(), w2.discharge()

In [ ]:
# 2 head specified wells
wh1.discharge(), wh2.discharge()

In [ ]:
# WellString, note that Q is returned in layers with well screens
ws.discharge().T

Compare heads

In [ ]:
# 2 discharge wells
w1.headinside(), w2.headinside()

In [ ]:
# 2 HeadWells, specified, not computed
wh1.headinside(), wh2.headinside()

In [ ]:
ws.wlist[0].headinside(), ws.wlist[1].headinside()

## HeadWellString

A series of connected wells with equal head (pressure) inside the well and a specified
head at a given point $(x_c, y_c)$. By default the specified head is applied in layer
0, but this can be set with `lc=<layer>`in the HeadWellString element.

### Single layer model

In [ ]:
# model parameters
kh = 10  # m/day

ctop = 1000.0  # resistance top leaky layer in days

ztop = 0.0  # surface elevation
zbot = -20.0  # bottom elevation of the model

z = [1.0, ztop, zbot]
kaq = np.array([kh])
c = np.array([ctop])

# point at which head is specified
xc = 10.0
yc = 0.0
hw = -2.0  # specified head at (xc, yc)

Build a single layer model with two HeadWells.

_Note: we need to nudge the control point for one of the wells here to avoid a singular
matrix for solving the system._

In [ ]:
mlref4 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
wh1 = tml.HeadWell(mlref4, 0, -10, hw=hw, rw=0.1, xc=xc, yc=yc)
wh2 = tml.HeadWell(mlref4, 0, 10, hw=hw, rw=0.1, xc=xc, yc=yc - 1e-6)  # nudge control pt
mlref4.solve()

Create model with a HeadWellString.

In [ ]:
ml2 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
ws = tml.HeadWellString(ml2, [(0, -10), (0, 10)], hw=-2, rw=0.1, xc=10, yc=0)
ml2.solve()

Plot the head contours for both models.

In [ ]:
ilay = 0
levels = [-2.4, -2.2, -2.0, -1.8, -1.6, -1.4, -1.2]
mlref4.plots.contour(
    win=(-50, 50, -50, 50),
    ngr=51,
    levels=levels,
    decimals=2,
    layers=[ilay],
)
plt.plot(w1.xc, w1.yc, "kx")
ml2.plots.contour(
    win=(-50, 50, -50, 50),
    ngr=51,
    levels=levels,
    decimals=2,
    layers=[ilay],
    newfig=False,
    linestyles="dashed",
    color="C1",
);

Plot the heads along y for both models.

In [ ]:
y = np.linspace(-50, 50, 101)
x = np.zeros_like(y)
href = mlref4.headalongline(x, y)
h2 = ml2.headalongline(x, y)

ilay = 0
plt.figure(figsize=(10, 2))
plt.plot(y, href[ilay], label="Reference HeadWells")
plt.plot(y, h2[ilay], "k.", label="HeadWellString model")
plt.ylabel("head [m]")
plt.xlabel("y [m]")
plt.legend(loc=(0, 1), frameon=False, ncol=2);

Plot the head along y at x=10, to show that the specified head is met

In [ ]:
y = np.linspace(-50, 50, 101)
x = ws.xc * np.ones_like(y)
href = mlref4.headalongline(x, y)
h2 = ml2.headalongline(x, y)

ilay = 0
plt.figure(figsize=(10, 2))
plt.plot(y, href[ilay], label="Reference HeadWells")
plt.plot(y, h2[ilay], "--", label="HeadWellString model")
plt.plot(ws.yc, ws.hw, "kx", label="Specified head")
plt.ylabel("head (at x=10) [m]")
plt.xlabel("y [m]")
plt.legend(loc=(0, 1), frameon=False, ncol=3);

Compare discharges

In [ ]:
# 2 HeadWells
wh1.discharge(), wh2.discharge()

In [ ]:
# HeadWellString model
ws.discharge()

Compare heads inside the well

In [ ]:
# 2 Headwells
wh1.headinside(), wh2.headinside()

In [ ]:
# HeadWellString
ws.wlist[0].headinside(), ws.wlist[1].headinside()

## Multi-layer example

An example of a HeadWellString in a multi-layer model.

In [ ]:
# model parameters
kh = [5, 10, 20]  # m/day

c = [1000.0, 100.0, 1.0]  # resistance leaky layers in days

ztop = 0.0  # surface elevation
zbot = -50.0  # bottom elevation of the model

z = [1.0, ztop, -10, -15, -25, -26, zbot]
kaq = np.array(kh)
c = np.array(c)

Create a multi-layer model with two HeadWells.

In [ ]:
mlref5 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
wh1 = tml.HeadWell(mlref5, 0, -10, hw=hw, rw=0.1, layers=[0, 1, 2], xc=xc, yc=yc)
wh2 = tml.HeadWell(
    mlref5, 0, 10, hw=hw, rw=0.1, layers=[0, 1, 2], xc=xc, yc=yc - 1e-6
)  # nudge control pt to avoid singular matrix in solve
mlref5.solve()

Create a model with a HeadWellString.

In [ ]:
ml3 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
ws = tml.HeadWellString(
    ml3, xy=[(0, -10), (0, 10)], hw=hw, rw=0.1, layers=[0, 1, 2], xc=xc, yc=yc
)
ml3.solve()

Plot the head contours in the reference model and the HeadWellString model.

Note that these models do not compute the exact same thing. The HeadWellString requires
the head to be equal to -2 in layer 0 at $(10, 0)$ whereas the model with 2 HeadWells
requires the Head to be equal to -2 in each layer at $(10, 0)$.

In [ ]:
ilay = 0
levels = [-2.4, -2.2, -2.0, -1.8, -1.6, -1.4, -1.2]
mlref5.plots.contour(
    win=(-50, 50, -50, 50),
    ngr=51,
    levels=levels,
    decimals=2,
    layers=[ilay],
)
plt.plot(w1.xc, w1.yc, "kx")
ml3.plots.contour(
    win=(-50, 50, -50, 50),
    ngr=51,
    levels=levels,
    decimals=2,
    layers=[ilay],
    newfig=False,
    linestyles="dashed",
    color="C1",
);

Plot the head along y in the reference model and the HeadWellString model.

In [ ]:
y = np.linspace(-50, 50, 101)
x = np.zeros_like(y)
href5 = mlref5.headalongline(x, y)
h3 = ml3.headalongline(x, y)

ilay = 1
plt.figure(figsize=(10, 2))
plt.plot(y, href5[ilay], label="Reference HeadWells")
plt.plot(y, h3[ilay], "--", label="HeadWellString model")
plt.ylabel("head [m]")
plt.xlabel("y [m]")
plt.legend(loc=(0, 1), frameon=False, ncol=3);

In [ ]:
y = np.linspace(-50, 50, 101)
x = ws.xc * np.ones_like(y)
href5 = mlref5.headalongline(x, y)
h3 = ml3.headalongline(x, y)

ilay = 0
plt.figure(figsize=(10, 2))
plt.plot(y, href5[ilay], label="Reference HeadWells")
plt.plot(y, h3[ilay], "--", label="HeadWellString model")
plt.plot(ws.yc, ws.hw, "kx", label="Specified head")
plt.ylabel("head (at x=10) [m]")
plt.xlabel("y [m]")
plt.legend(loc=(0, 1), frameon=False, ncol=3);

Compare discharges

In [ ]:
# 2 HeadWells
wh1.discharge(), wh2.discharge()

In [ ]:
# HeadWellString
ws.discharge().T

Compare heads inside the well

In [ ]:
# Separate wells
wh1.headinside(), wh2.headinside()

In [ ]:
# HeadWellString
ws.wlist[0].headinside(),ws.wlist[1].headinside()

Compare heads at specified point

In [ ]:
# 2 HeadWells
mlref5.head(xc, yc)

In [ ]:
# HeadWellString
ml3.head(xc, yc)

### Different layers per well

The WellString elements allow the specification of different layers per well, as shown
in the example below.

In [ ]:
ml4 = tml.ModelMaq(kaq=kaq, c=c, z=z, topboundary="semi", hstar=0)
ws = tml.HeadWellString(
    ml4, xy=[(0, -10), (0, 10)], hw=-1, rw=0.1, layers=[(1,), (2,)], xc=10, yc=0
)
ml4.solve()

Plot the wells in a cross-section.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3))
ml4.plots.xsection(xy=[(0, 20), (0, -20)], labels=True, ax=ax)
for w in ws.wlist:
    for ilay in w.layers:
        ax.plot([w.yc, w.yc], [ml4.aq.zaqtop[ilay], ml4.aq.zaqbot[ilay]], "k-")

Plot a cross section of the head along y, showing that the head in layer 0 runs through
-1, the head we specified in the HeadWellString.

In [ ]:
y = np.linspace(-50, 50, 101)
x = ws.xc * np.ones_like(y)
h4 = ml4.headalongline(x, y)

ilay = 0
plt.figure(figsize=(10, 2))
plt.plot(y, h4[ilay], "-", label="HeadWellString model")
plt.plot(ws.yc, ws.hw, "kx", label="Specified head")
plt.ylabel("head (at x=10) [m]")
plt.xlabel("y [m]")
plt.legend(loc=(0, 1), frameon=False, ncol=3);

Get discharge per well (note that this discharge is for different layers):

In [ ]:
ws.discharge()

Show that head inside the wells is equal:

In [ ]:
ws.wlist[0].headinside(), ws.wlist[1].headinside()